# Set Required Args to Activate Modules

In [1]:
# from workflow.scripts.resources import Resources

# single resource

In [2]:
# # Iterate over provinces for both solar and wind resources
# resource_types = ['wind', 'solar']
# provinces=['BC']  #'AB','SK','ON','NS'
# for province_code in provinces:
#     for resource_type in resource_types:
#         required_args = {
#             "config_file_path": 'config/config.yml',
#             "province_short_code": province_code,
#             "resource_type": resource_type
#         }
        
#         # Create an instance of Resources and execute the module
#         resource_module = Resources(**required_args)
#         resource_module.execute_module()
        

In [4]:
import geopandas as gpd
import pandas as pd
import dask.dataframe as dd
import dask_geopandas as dgpd
from linkingtool.hdf5_handler import DataHandler
store=DataHandler('data/store/resources_BC.h5')
from linkingtool.gwa import GWACells
gwa=GWACells(resource_type='wind')

>> Store initialized with the given path: data/store/resources_BC.h5
>> Store initialized with the given path: data/store/resources_BC.h5


In [ ]:
boundary1.clip(dgd1)

TypeError: If 'mask' is list-like, it must have four values (minx, miny, maxx, maxy)